# Import Libraries

In [1]:
import pandas as pd
import numpy as np

# Some simple functions to generate synthetic or dummy data

In [2]:
def produce_synthetic_game_log(T, L, q_max):
    game_log = pd.DataFrame({'t' : [t for t in range(0, T)]})
    for l in range(L):
        game_log['q_' + str(l)] = [np.random.randint(q_max) for _ in range(0, T)]
    return game_log

def produce_database(number_of_games):
    return pd.concat([produce_synthetic_game_log(T, L, q_max) for _ in range(number_of_games)])

## Parameters of the synthetic log we want to produce

In [3]:
T = 10
L = 5
q_max = 5
database = produce_database(10000)
database

,t,q_0,q_1,q_2,q_3,q_4
0,0,2,1,4,2,2
1,1,4,2,0,4,3
2,2,0,2,1,4,0
3,3,3,4,0,1,0
4,4,0,1,3,1,1
5,5,4,3,2,3,1
6,6,3,1,4,0,1
7,7,3,1,3,2,2
8,8,4,4,1,3,4
9,9,0,4,1,2,4


## Now, we fix a level and a time step, and clean the data accordingly

In [4]:
current_level = 3
cols = {'q_' + str(i) for i in range(0, L)}
curr_level_col_name = 'q_' + str(current_level)
prev_level_col_name = 'q_' + str(current_level - 1)
drop_cols = cols - {curr_level_col_name, prev_level_col_name}

current_time = 1
time_steps = {t for t in range(0, T)}
drop_time_steps = time_steps - {current_time, current_time - 1}

# Select only the data that we need for this computation
y = database.drop(drop_time_steps, axis=0).drop(drop_cols, axis = 1)
y

,t,q_2,q_3
0,0,4,2
1,1,0,4
0,0,2,1
1,1,1,2
0,0,4,3
1,1,1,2
0,0,0,2
1,1,4,4
0,0,0,2
1,1,3,3


## Generate 3 columns: Q_{t-1}^{l-1}, Q_{t-1}^{l}, and Q_{t}^{l}

In [5]:
# New column names for the mesh, we care about:
# previous level, previous time step -> Q_{t-1}^{l-1}
# current level, previous time step  -> Q_{t-1}^{l}
# current level, current time step   -> Q_{t}^{l}
prev_level_prev_time_col_name = prev_level_col_name + '_t_' + str(current_time - 1)
curr_level_prev_time_col_name = curr_level_col_name + '_t_' + str(current_time - 1)
curr_level_curr_time_col_name = curr_level_col_name + '_t_' + str(current_time)

# Horizontal merger of the next time step data
w = pd.concat([
            y.drop(current_time).reset_index(drop=True).
                rename(columns={prev_level_col_name : prev_level_prev_time_col_name,
                                curr_level_col_name : curr_level_prev_time_col_name}), 
            y.drop(current_time - 1).reset_index(drop=True).
                rename(columns={curr_level_col_name : curr_level_curr_time_col_name})
              ], axis=1).drop(['t', prev_level_col_name], axis=1)
w

,q_2_t_0,q_3_t_0,q_3_t_1
0,4,2,4
1,2,1,2
2,4,3,2
3,0,2,4
4,0,2,3
5,0,4,0
6,0,3,4
7,0,4,1
8,2,2,3
9,2,2,1


# For each possible value Q_{t}^{l} \in {0, ..., q_max}, count

In [6]:
# Count occurrences from 0 to q_max.
# More specifically, for each value q \in {0, ..., q_max}, create a column with a 1 if 
# the current row coincides with q, and 0 otherwise. 
for i in range(0, q_max):
    w[curr_level_col_name + '_t_' + str(current_time) + '=' + str(i)] = \
            np.where(w[curr_level_col_name + '_t_' + str(current_time)] == i, 1, 0)
w = w.drop(curr_level_col_name + '_t_' + str(current_time), axis=1)
w

,q_2_t_0,q_3_t_0,q_3_t_1=0,q_3_t_1=1,q_3_t_1=2,q_3_t_1=3,q_3_t_1=4
0,4,2,0,0,0,0,1
1,2,1,0,0,1,0,0
2,4,3,0,0,1,0,0
3,0,2,0,0,0,0,1
4,0,2,0,0,0,1,0
5,0,4,1,0,0,0,0
6,0,3,0,0,0,0,1
7,0,4,0,1,0,0,0
8,2,2,0,0,0,1,0
9,2,2,0,1,0,0,0


## The final magic here... group by the two variables: Q_{t-1}^{l-1}, Q_{t-1}^{l} and sum up

In [7]:
# Computing histograms
z = w.groupby([prev_level_prev_time_col_name, curr_level_prev_time_col_name]).sum()
z

q_3_t_1=0  q_3_t_1=1  q_3_t_1=2  q_3_t_1=3  q_3_t_1=4
q_2_t_0 q_3_t_0                                                       
0       0               83         74         70         83         80
        1               84         85         73         86         91
        2               77         73         81         83         73
        3               69         79         78         66         79
        4               87         82         78         72         85
1       0               80         72         95         72         83
        1               77         82         71         77         73
        2               85         75         78        103         82
        3               80         81         90         90         91
        4               76         79         77         70         75
2       0               71         76         72         80         70
        1               84         83         93         82         82
        2               75         85         75         66         88
        3               75         84         73         83         82
        4               84         84         65         70         94
3       0               75         65         66         80         89
        1               98         78         93         97         72
        2               88         80         69         80         91
        3               81         89         75         70         86
        4               71         86         75         84         88
4       0               88         88         70         79         70
        1               92         82         81         85         86
        2               80         72         80         79         87
        3               74         77         71         97         90
        4               74         66         85         82         78

# Normalization into a probability distribution

In [8]:
final = z.div(z.sum(axis=1), axis=0)
final

q_3_t_1=0  q_3_t_1=1  q_3_t_1=2  q_3_t_1=3  q_3_t_1=4
q_2_t_0 q_3_t_0                                                       
0       0         0.212821   0.189744   0.179487   0.212821   0.205128
        1         0.200477   0.202864   0.174224   0.205251   0.217184
        2         0.198966   0.188630   0.209302   0.214470   0.188630
        3         0.185984   0.212938   0.210243   0.177898   0.212938
        4         0.215347   0.202970   0.193069   0.178218   0.210396
1       0         0.199005   0.179104   0.236318   0.179104   0.206468
        1         0.202632   0.215789   0.186842   0.202632   0.192105
        2         0.200946   0.177305   0.184397   0.243499   0.193853
        3         0.185185   0.187500   0.208333   0.208333   0.210648
        4         0.201592   0.209549   0.204244   0.185676   0.198939
2       0         0.192412   0.205962   0.195122   0.216802   0.189702
        1         0.198113   0.195755   0.219340   0.193396   0.193396
        2         0.192802   0.218509   0.192802   0.169666   0.226221
        3         0.188917   0.211587   0.183879   0.209068   0.206549
        4         0.211587   0.211587   0.163728   0.176322   0.236776
3       0         0.200000   0.173333   0.176000   0.213333   0.237333
        1         0.223744   0.178082   0.212329   0.221461   0.164384
        2         0.215686   0.196078   0.169118   0.196078   0.223039
        3         0.201995   0.221945   0.187032   0.174564   0.214464
        4         0.175743   0.212871   0.185644   0.207921   0.217822
4       0         0.222785   0.222785   0.177215   0.200000   0.177215
        1         0.215962   0.192488   0.190141   0.199531   0.201878
        2         0.201005   0.180905   0.201005   0.198492   0.218593
        3         0.180929   0.188264   0.173594   0.237164   0.220049
        4         0.192208   0.171429   0.220779   0.212987   0.202597

# Save to .csv file. This saves a SINGLE conditional probability table...

In [9]:
final.to_csv('data/T_' + str(T) + '_L_' + str(L) + '_' + curr_level_curr_time_col_name + '.csv')